In [1]:
import os                                                 # to set current working directory 
import numpy as np                                        # arrays and matrix math
import pandas as pd                                       # DataFrames
import matplotlib.pyplot as plt                           # plotting
cmap = plt.cm.inferno                                     # color map
import geostatspy.geostats as geostats
import geostatspy.GSLIB as GSLIB
import math # For n_effective calculations
import scipy # For n_effective calculations
import time
import json

from scipy.integrate import simps
from scipy.stats import norm
from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from IPython.display import clear_output

%matplotlib inline

In [2]:
def add_intersection_points(x, y):
    new_x = []
    new_y = []
    
    for i in range(len(y)):
        new_x.append(x[i])
        new_y.append(y[i])
        
        if i < len(y) - 1 and np.sign(y[i]) != np.sign(y[i + 1]):
            # Sign change detected
            x1, x2 = x[i], x[i + 1]
            y1, y2 = y[i], y[i + 1]
            
            # Calculate the x value where the line intersects the x-axis (y = 0)
            x_intersection = x1 - (y1 * (x2 - x1)) / (y2 - y1)
            
            # Add the intersection point to the new arrays
            new_x.append(x_intersection)
            new_y.append(0)
    
    return np.array(new_x), np.array(new_y)

def goodness(p_intervals, fraction_in, return_plots = True, return_areas = True):
    # Example dataset (replace these with your actual data)
    x = p_intervals
    y = fraction_in - p_intervals

    x, y = add_intersection_points(x, y)

    # Split the dataset into positive and negative parts of y
    positive_mask = y > 0
    negative_mask = y < 0

    x_positive = x[positive_mask]
    y_positive = y[positive_mask]

    x_negative = x[negative_mask]
    y_negative = y[negative_mask]

    # Perform integration on the positive and negative parts separately
    try:
        area_positive = simps(y_positive, x_positive)
    except:
        area_positive = 0

    try:
        area_negative = simps(y_negative, x_negative)
    except:
        area_negative = 0
        
    if return_plots:
        # Plot the original data and highlight positive and negative parts
        plt.figure(figsize=(10, 6))
        plt.plot(x, y, label='Original Data', color='blue')
        plt.fill_between(x_positive, y_positive, color='green', alpha=0.5, label='Positive Area')
        plt.fill_between(x_negative, y_negative, color='red', alpha=0.5, label='Negative Area')
        plt.xlabel('p')
        plt.ylabel('frac-p')
        plt.legend()
        plt.title('Accurracy Plot')
        plt.grid(True)

        #Test 0 points
        plt.scatter(x[y==0], y[y==0])

        # Show the plot
        plt.show()
    
    if return_areas:
        print("Integral of the positive part:", area_positive)
        print("Integral of the negative part:", abs(area_negative))
        
    G = 1 - (1 * area_positive + 2 * abs(area_negative))
    return G

def goodness2(p_intervals, fraction_in, return_plots=True, return_areas=True):
    # Example dataset (replace these with your actual data)
    x = p_intervals
    y = fraction_in - p_intervals
    
    # x, y = add_intersection_points(x, y)

    # Calculate the area under the curve (y-values)
    try:
        area = simps(y, x)
    except:
        return -1
    
    if return_plots:
        # Plot the original data
        plt.figure(figsize=(10, 6))
        plt.plot(x, y, label='Original Data', color='blue')
        plt.xlabel('p')
        plt.ylabel('frac-p')
        plt.legend()
        plt.title('Accuracy Plot')
        plt.grid(True)
        
        # Show the plot
        plt.show()
    
    if return_areas:
        print("Integral of the curve:", area)
        
    G = 1 - abs(area)
    return G

In [ ]:
import os
import pandas as pd
import numpy as np
import json
from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import norm
from IPython.display import clear_output
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')




# Define the directory path
directory = r"dataset"

# Get all items in the directory
items = os.listdir(directory)

# Filter out only directories
filename_list = [item for item in items if os.path.isdir(os.path.join(directory, item))]

for filenum, filename in enumerate(filename_list):
    #### READ DATA AND PARSE
    df_path = os.path.join(directory, filename, filename + '.csv')
    if not os.path.exists(df_path):
        print(f"Data file for {filename} not found. Skipping.")
        continue
    df = pd.read_csv(df_path)
    
    # We are sampling every 500 meters
    df = df.iloc[::5] 
    df = df.sample(n=500, random_state=73073)
    predictors = df.iloc[:, 2:-1].columns.values
    response = df.columns[-1]
    
    df_train, df_test = train_test_split(df, test_size=0.5, random_state=73073)
    X_train = df_train.loc[:, predictors].values
    y_train = X_train.mean(axis=1)
    X_test = df_test.loc[:, predictors].values
    y_test = X_test.mean(axis=1)
    
    # Standardizing the predictor variables
    scaler_X = StandardScaler()
    X_train = scaler_X.fit_transform(X_train)
    X_test = scaler_X.transform(X_test)

    # Set constants
    max_depth_constant = 3
    max_leaf_nodes_constant = 3
    n_estimators_constant = 100
    num_features = X_train.shape[1]  # Number of features, which should be 9

    # Initialize parameters for the spatial bootstrap with bagging
    L = n_estimators_constant  # Number of estimators
    L_2 = 10  # Number of outer iterations
    n_bins = 50  # Number of bins for histograms

    print(f"Processing file {filenum + 1}/{len(filename_list)}: {filename}")
    json_path = os.path.join(directory, filename, filename + '_var_params_and_n_eff.json')
    if not os.path.exists(json_path):
        print(f"JSON file with n_eff for {filename} not found. Skipping.")
        continue
    with open(json_path, 'r') as f:
        data = json.load(f)

    #### ASSIGN VARIABLES FROM PARAMS
    n_eff_calculated = data['n_eff_calculated']

    # Create a list of max_samples values from 10 to 1000 in increments of 10, including n_eff_calculated
    max_samples_list = list(range(10, 251, 20))
    if n_eff_calculated not in max_samples_list:
        max_samples_list.append(n_eff_calculated)
        max_samples_list = sorted(max_samples_list)

    # Ensure epsilon is defined
    epsilon = 1e-10

    ### BAGGING APPROACH WITH SENSITIVITY ON max_samples ###
    print(f"Starting Bagging sensitivity analysis on max_samples")
    # Initialize dictionaries to store results for Bagging
    bagging_results = {}

    # Main Loop for Bagging with varying max_samples
    for max_samples_value in max_samples_list:
        if max_samples_value > len(X_train):
            print(f"Bagging: max_samples {max_samples_value} exceeds training data size. Skipping.")
            continue
        print(f"Bagging with max_samples = {max_samples_value}")

        # Define base estimator with fixed max_depth and max_leaf_nodes
        base_regressor = DecisionTreeRegressor(
            max_depth=max_depth_constant,
            max_leaf_nodes=max_leaf_nodes_constant,
            random_state=73073
        )

        # Create BaggingRegressor
        bagging_regressor = BaggingRegressor(
            base_estimator=base_regressor,
            n_estimators=L,
            max_samples=max_samples_value,
            bootstrap=True,
            random_state=73073,
            n_jobs=-1
        )
        bagging_regressor.fit(X_train, y_train)
        estimators = bagging_regressor.estimators_

        # Access predictions
        reals = np.array([est.predict(X_test) for est in estimators])
        trains = np.array([est.predict(X_train) for est in estimators])

        # Aggregate predictions
        aggregate_test = np.mean(reals, axis=0)
        aggregate_train = np.mean(trains, axis=0)

        st_dev_test = np.std(reals, axis=0)
        st_dev_train = np.std(trains, axis=0)

        # Handle zero standard deviation
        st_dev_test[st_dev_test == 0] += epsilon
        st_dev_train[st_dev_train == 0] += epsilon

        # Calculate cdf values for test data
        cdf_values_test = norm.cdf(y_test, loc=aggregate_test, scale=st_dev_test)
        # Calculate cdf values for train data
        cdf_values_train = norm.cdf(y_train, loc=aggregate_train, scale=st_dev_train)

        bins = 100
        fraction_in_test = np.zeros(bins)
        fraction_in_train = np.zeros(bins)
        p_intervals = np.linspace(0.0, 1.0, bins)

        for i, p in enumerate(p_intervals):
            # Test data
            test_result_test = (cdf_values_test > 0.5 - 0.5 * p) & (cdf_values_test < 0.5 + 0.5 * p)
            fraction_in_test[i] = test_result_test.sum() / len(cdf_values_test)
            # Train data
            test_result_train = (cdf_values_train > 0.5 - 0.5 * p) & (cdf_values_train < 0.5 + 0.5 * p)
            fraction_in_train[i] = test_result_train.sum() / len(cdf_values_train)

        # Calculate goodness metrics
        goodness_test = goodness(p_intervals, fraction_in_test, return_plots=False, return_areas=False)
        goodness_train = goodness(p_intervals, fraction_in_train, return_plots=False, return_areas=False)
        goodness2_test = goodness2(p_intervals, fraction_in_test, return_plots=False, return_areas=False)
        goodness2_train = goodness2(p_intervals, fraction_in_train, return_plots=False, return_areas=False)

        # Calculate MSE
        mse_test = mean_squared_error(y_test, aggregate_test)
        mse_train = mean_squared_error(y_train, aggregate_train)

        # Store results in the dictionary
        bagging_results[str(max_samples_value)] = {
            'mse_train': mse_train,
            'mse_test': mse_test,
            'goodness_train': goodness_train,
            'goodness_test': goodness_test,
            'goodness2_train': goodness2_train,
            'goodness2_test': goodness2_test,
            'aggregate_train': aggregate_train,
            'aggregate_test': aggregate_test,
            'fractions_train': fraction_in_train,
            'fractions_test': fraction_in_test,
            'reals_train': trains,
            'reals_test': reals
        }

    # Create a directory to store the results for this filename
    results_dir = os.path.join(directory, filename, 'results_max_samples')
    os.makedirs(results_dir, exist_ok=True)

    # Save the bagging results using NumPy's npz format
    np.savez_compressed(
        os.path.join(results_dir, filename + '_bagging_results.npz'),
        bagging_results=bagging_results
    )

    ### RANDOM FOREST WITH SENSITIVITY ON max_samples AND max_features ###
    print(f"Starting Random Forest sensitivity analysis on max_samples and max_features")

    # Initialize dictionary to store Random Forest results
    rf_results = {}

    # Range of max_features to test (from 1 to num_features)
    max_features_list = list(range(1, num_features + 1))

    # Loop over max_samples values
    for max_samples_value in max_samples_list:
        if max_samples_value > len(X_train):
            print(f"Random Forest: max_samples {max_samples_value} exceeds training data size. Skipping.")
            continue
        print(f"Random Forest with max_samples = {max_samples_value}")

        # Initialize dictionary for current max_samples_value
        rf_results[str(max_samples_value)] = {}

        # Loop over max_features values
        for max_features_value in max_features_list:
            print(f"  max_features = {max_features_value}")

            # Initialize RandomForestRegressor with varying max_samples and max_features
            rf_regressor = RandomForestRegressor(
                n_estimators=n_estimators_constant,
                max_depth=max_depth_constant,
                max_leaf_nodes=max_leaf_nodes_constant,
                max_features=max_features_value,  # Varying max_features
                max_samples=max_samples_value,
                random_state=73073,
                n_jobs=-1  # Use all available cores
            )

            # Fit the model on the training data
            rf_regressor.fit(X_train, y_train)

            # Predict on test and train data
            y_pred_test_rf = rf_regressor.predict(X_test)
            y_pred_train_rf = rf_regressor.predict(X_train)

            # Calculate MSE
            mse_test_rf = mean_squared_error(y_test, y_pred_test_rf)
            mse_train_rf = mean_squared_error(y_train, y_pred_train_rf)

            # Extract individual tree predictions
            reals_rf_test = np.array([tree.predict(X_test) for tree in rf_regressor.estimators_])
            reals_rf_train = np.array([tree.predict(X_train) for tree in rf_regressor.estimators_])

            # Aggregate predictions
            aggregate_rf_test = np.mean(reals_rf_test, axis=0)
            aggregate_rf_train = np.mean(reals_rf_train, axis=0)

            st_dev_rf_test = np.std(reals_rf_test, axis=0)
            st_dev_rf_train = np.std(reals_rf_train, axis=0)

            # Handle zero standard deviation
            st_dev_rf_test[st_dev_rf_test == 0] += epsilon
            st_dev_rf_train[st_dev_rf_train == 0] += epsilon

            # Calculate cdf values
            cdf_values_rf_test = norm.cdf(y_test, loc=aggregate_rf_test, scale=st_dev_rf_test)
            cdf_values_rf_train = norm.cdf(y_train, loc=aggregate_rf_train, scale=st_dev_rf_train)

            bins = 100
            fraction_in_rf_test = np.zeros(bins)
            fraction_in_rf_train = np.zeros(bins)
            p_intervals = np.linspace(0.0, 1.0, bins)

            for i, p in enumerate(p_intervals):
                # Test data
                test_result_rf_test = (cdf_values_rf_test > 0.5 - 0.5 * p) & (cdf_values_rf_test < 0.5 + 0.5 * p)
                fraction_in_rf_test[i] = test_result_rf_test.sum() / len(cdf_values_rf_test)
                # Train data
                test_result_rf_train = (cdf_values_rf_train > 0.5 - 0.5 * p) & (cdf_values_rf_train < 0.5 + 0.5 * p)
                fraction_in_rf_train[i] = test_result_rf_train.sum() / len(cdf_values_rf_train)

            # Calculate goodness metrics
            goodness_rf_test = goodness(p_intervals, fraction_in_rf_test, return_plots=False, return_areas=False)
            goodness_rf_train = goodness(p_intervals, fraction_in_rf_train, return_plots=False, return_areas=False)
            goodness2_rf_test = goodness2(p_intervals, fraction_in_rf_test, return_plots=False, return_areas=False)
            goodness2_rf_train = goodness2(p_intervals, fraction_in_rf_train, return_plots=False, return_areas=False)

            # Store results under current max_features_value
            rf_results[str(max_samples_value)][str(max_features_value)] = {
                'mse_train': mse_train_rf,
                'mse_test': mse_test_rf,
                'goodness_train': goodness_rf_train,
                'goodness_test': goodness_rf_test,
                'goodness2_train': goodness2_rf_train,
                'goodness2_test': goodness2_rf_test,
                'aggregate_train': aggregate_rf_train,
                'aggregate_test': aggregate_rf_test,
                'fractions_train': fraction_in_rf_train,
                'fractions_test': fraction_in_rf_test,
                'reals_train': reals_rf_train,
                'reals_test': reals_rf_test
            }

    # Save Random Forest results
    rf_results_dir = os.path.join(directory, filename, 'rf_results_max_samples_features')
    os.makedirs(rf_results_dir, exist_ok=True)

    # Save the rf_results using NumPy's npz format
    np.savez_compressed(
        os.path.join(rf_results_dir, filename + '_rf_results.npz'),
        rf_results=rf_results
    )

    print(f"Processing for {filename} completed.")
    print(f"Processed {filenum + 1}/{len(filename_list)} files.")
    clear_output(wait=True)


Data file for vardump not found. Skipping.
